# How to Become a Data Scientist
***

**Jack Cook — 11/08/17**

Kaggle recently conducted a survey to gauge the state of data science and machine learning. They received nearly 17,000 responses, and left us with a lot to analyze. Rather than finding summary statistics about the survey's respondents, I figured I'd try to specifically see how these results could be useful to people like me, who are figuring out how to get started.

If you're not that interested in the analysis, and just want to **see the collective recommendations of 17,000 industry professionals**, feel free to [click here](#conclusion) to skip to the end. Otherwise, stick around to see how I reached these conclusions.

# Table of Contents

* [1. Importing and preprocessing](#importing_and_preprocessing)
  * [1.1. Importing data](#importing_data)
  * [1.2. Data preprocessing](#data_preprocessing)
  * [1.3. Theme](#theme)
* [2. Getting started](#getting_started)
  * [2.1. Programming language](#programming_language)
  * [2.2. Majors and likely positions](#majors_and_likely_positions)
* [3. What to learn about](#what_to_learn_about)
  * [3.1. Resources for learning](#resources_for_learning)
  * [3.2. Important job skills](#important_job_skills)
  * [3.3. Tools in the real world](#tools_in_the_real_world)
  * [3.4. Important job methods](#important_job_methods)
* [4. Conclusion](#conclusion)

# 1. Importing and Preprocessing
<a id="importing_and_preprocessing"></a>

## 1.1. Importing data
<a id="importing_data"></a>

In [ ]:
library(data.table)
library(dplyr, warn.conflicts = FALSE)
library(ggplot2)
library(tibble)

results <- as.tibble(suppressWarnings(fread("../input/multipleChoiceResponses.csv")))

## 1.2. Data preprocessing
<a id="data_preprocessing"></a>

Many of the job titles and majors that are referred to throughout this analysis have very long names. I reduced the lengths of a few of them to make the graphs more readable, removed illegal characters from two work methods, and converted job satisfaction into a numeric.

In [ ]:
results$CurrentJobTitleSelect[results$CurrentJobTitleSelect == "Software Developer/Software Engineer"] <- "Software Engineer"
results$CurrentJobTitleSelect[results$CurrentJobTitleSelect == "Operations Research Practitioner"] <- "Operations Research"

results$MajorSelect[results$MajorSelect == "Engineering (non-computer focused)"] <- "Engineering"
results$MajorSelect[results$MajorSelect == "Information technology, networking, or system administration"] <- "Information technology"
results$MajorSelect[results$MajorSelect == "Management information systems"] <- "Information systems"

results_names <- names(results)
results_names[results_names == "WorkMethodsFrequencyA/B"] <- "WorkMethodsFrequencyABTesting"
results_names[results_names == "WorkMethodsFrequencyCross-Validation"] <- "WorkMethodsFrequencyCrossValidation"
names(results) <- results_names

results$JobSatisfaction <- suppressWarnings(as.numeric(substr(results$JobSatisfaction, start = 1, stop = 2)))

## 1.3. Theme
<a id="theme"></a>

I have multiple data displays throughout the analysis, and I think there's something to be said about making graphs look nice. I'll be building off of this theme for most of my graphs.

In [ ]:
jack_theme <- theme(
    plot.background = element_rect(fill = "#eeeeee"),
    panel.background = element_rect(fill = "#eeeeee"),
    legend.background = element_rect(fill = "#eeeeee"),
    legend.title = element_text(size = 12, family = "Helvetica", face = "bold"),
    legend.text = element_text(size = 9, family = "Helvetica"),
    panel.grid.major = element_line(size = 0.4, linetype = "solid", color = "#cccccc"),
    panel.grid.minor = element_line(size = 0),
    plot.title = element_text(size = 20, family = "Helvetica", face = "bold", hjust = 0.5, margin = margin(b = 20)),
    axis.title = element_text(size = 14, family = "Helvetica", face = "bold"),
    axis.title.x = element_text(margin = margin(t = 20)),
    axis.title.y = element_text(margin = margin(r = 20)),
    axis.ticks = element_blank(),
    plot.margin = unit(c(1, 1, 1, 1), "cm")
)

# 2. Getting started
<a id="getting_started"></a>

## 2.1. Programming language
<a id="programming_language"></a>

One of the first questions many beginner data scientists will ask themselves is what programming language they should learn. In this graph, we can see how Python and R stacked up against each other based on recommendations by participents with each job title.

In [ ]:
results %>%
    rename(Language = LanguageRecommendationSelect, title = CurrentJobTitleSelect) %>%
    filter(Language == "R" | Language == "Python") %>%
    filter(title != "") %>%
    group_by(title, Language) %>%
    count() %>%
    ggplot(aes(title, n, color = Language)) +
    ggtitle("Python vs. R by Job Title") +
    labs(x = "Job Title", y = "Count") +
    geom_point(size = 4) +
    jack_theme +
    theme(
        axis.text.x = element_text(angle = 330, hjust = 0),
        axis.title.x = element_text(margin = margin(t = 12))
    )

As we can see, **Python is favored for beginners over R in almost every scenario**, and notably so for data scientists, software developers, and machine learning engineers. However, R is very slightly favored by statisticians. It is technically favored for operations researchers as well, but I would attribute this to a very small sample size.

## 2.2. Majors and likely positions
<a id="majors_and_likely_positions"></a>

I haven't started college yet, and as such, have not yet had to declare a major. However, intuitively I would think that some majors are much more likely than others to hold certain jobs. In this next graph, we can confirm or disprove my beliefs, seeing what proportion of each job title is held by people with a certain major.

In [ ]:
results %>%
    rename(Major = MajorSelect, title = CurrentJobTitleSelect) %>%
    filter(title != "", Major != "") %>%
    group_by(title, Major) %>%
    summarize(n = n()) %>%
    mutate(freq = n / sum(n) * 100) %>%
    ggplot(aes(x = title, y = freq, fill = Major, label = ifelse(freq > 8, round(freq), ""))) +
    ggtitle("Major vs. Job Title") +
    labs(x = "Job Title", y = "Frequency (%)") +
    geom_bar(stat = "identity", position = position_stack()) +
    geom_text(position = position_stack(vjust = 0.5)) +
    scale_y_continuous(expand = c(0, 0)) +
    jack_theme +
    theme(
        axis.title.x = element_text(margin = margin(t = 8)),
        axis.text.x = element_text(angle = 320, hjust = 0)
    )

My intuition was mostly right, majors end up being a pretty good predictor of what someone will do in the future. A few of these trends are obvious: computer science majors tend to become computer scientists, programmers, and software engineers, while math majors tend to become statisticians and predictive modelers. Most physics majors go into research, and non-CS engineering majors called themselves engineers.

What I personally like about this graph is, and this is true, **at least one person from each major is fulfilling every job title**. To me, this shows that no matter what you study in school, you can do what you want as long as the passion is there.

# 3. What to learn about
<a id="what_to_learn_about"></a>

## 3.1. Resources for learning
<a id="resources_for_learning"></a>

One of the survey's first questions asked participants how useful they thought certain resources are for learning about data science. For my next graph, I plotted each resource's usefulness against its popularity. Popularity is the number of people who responded to this question, and usefulness is a weighted average of the responses given and the number of respondents. Each person got either a weight of 1 for an answer of "very useful", 0.5 for an answer of "somewhat useful", or no weight at all if they said it wasn't useful. In this graph, we can not only see the most useful resources out there, but also if any resources are overhyped or underutilized.

In [ ]:
# Get all column names that begin with "LearningPlatformUsefulness"
platforms <- grep("^LearningPlatformUsefulness", names(results), value=T)

names <- c()
popularities <- c()
scores <- c()

for (platform in platforms) {
    usefulness <- results %>%
        group_by_(platform) %>%
        count()
    
    # Popularity = the number of people who responded to this question
    popularity <- usefulness[[2]][2] + usefulness[[2]][3] + usefulness[[2]][4]
    
    # Usefulness = a weighted average determining the usefulness of this platform
    score <- (usefulness[[2]][2] * 0 + usefulness[[2]][3] * 0.5 + usefulness[[2]][4] * 1) / popularity
    
    names <- c(names, gsub("LearningPlatformUsefulness", "", platform))
    popularities <- c(popularities, popularity)
    scores <- c(scores, score)
}

scores_df <- data.frame(
    Popularity = popularities,
    Usefulness = scores,
    Name = names
)

ggplot(scores_df, aes(x = Usefulness, y = Popularity)) +
    ggtitle("Effectiveness of Learning Methods") +
    geom_point() +
    geom_text(aes(label = Name, family = "Helvetica"), nudge_y = 200) +
    jack_theme

We can see a couple of results that are immediately obvious. Podcasts, newsletters, and conferences all scored the lowest as far as how useful they are for learning, while **Kaggle competitions, Stack Overflow, online courses, and projects scored the highest**. We can also see that while many people love watching YouTube tutorials and reading blogposts, these may not be the most effective ways to learn about data science.

## 3.2. Important job skills
<a id="important_job_skills"></a>

We can reuse the same method used to create the last graph to make another graph for the questions that asked which technologies were considered to be the most important on the job. Here, I plotted popularity against usefulness again to see which technologies are used the most in the real world.

In [ ]:
# Get all column names that begin with "JobSkillImportance" and end in a letter
platforms <- grep("^JobSkillImportance.*[A-z]$", names(results), value=T)

names <- c()
popularities <- c()
scores <- c()

for (platform in platforms) {
    usefulness <- results %>%
        group_by_(platform) %>%
        count()
    
    # Popularity = the number of people who responded to this question
    popularity <- usefulness[[2]][2] + usefulness[[2]][3] + usefulness[[2]][4]
    
    # Usefulness = a weighted average determining the usefulness of this platform
    score <- (usefulness[[2]][2] * 1 + usefulness[[2]][3] * 0.5 + usefulness[[2]][4] * 0) / popularity
    
    names <- c(names, gsub("JobSkillImportance", "", platform))
    popularities <- c(popularities, popularity)
    scores <- c(scores, score)
}

scores_df <- data.frame(
    Popularity = popularities,
    Usefulness = scores,
    Name = names
)

ggplot(scores_df, aes(x = Usefulness, y = Popularity)) +
    ggtitle("Important Skills on the Job") +
    geom_point() +
    geom_text(aes(label = Name, family = "Helvetica"), nudge_y = 12) +
    jack_theme

Interestingly enough, MOOCs scored the lowest in terms of usefulness, even though online courses scored the second highest in the last question. Therefore, while courses can be good for learning about data science, maybe other resources should be used in order to learn skills that are useful for your job. We can see that **knowledge of Python, advanced statistics, and vizualization tools ranked the highest** in terms of skills that are useful in getting data science jobs.

## 3.3. Tools in the real world
<a id="tools_in_the_real_world"></a>

One of the survey's questions asked respondents how often they use certain technologies at work. Instead of making a scatterplot for this one, I put them into a table with multiple columns for job titles, since the responses essentially measured popularity anyway. More weight was given to technologies that were reportedly used more often, and by more people.

In [ ]:
# Get all column names that begin with "WorkToolsFrequency" and end in a letter
platforms <- grep("^WorkToolsFrequency.*[A-z]$", names(results), value=T)
positions <- c("All", "Data Scientist", "Software Engineer", "Researcher", "Machine Learning Engineer")

technologies <- matrix("", 10, length(positions))
colnames(technologies) <- positions
rownames(technologies) <- 1:10

i <- 1

for (position in positions) {
    names <- c()
    popularities <- c()
    
    if (position == "All") {
        position_results <- results
    } else {
        position_results <- results %>% filter(CurrentJobTitleSelect == position)
    }
    
    for (platform in platforms) {
        usefulness <- position_results %>%
            group_by_(platform) %>%
            count()
        
        # Popularity = the number of people who responded to this question
        popularity <- usefulness[[2]][2] + usefulness[[2]][3] + usefulness[[2]][4] + usefulness[[2]][5]
        
        # Usefulness = a weighted average determining how much this tool was used
        score <- (usefulness[[2]][2] * 1 + usefulness[[2]][3] * 0.67 + usefulness[[2]][4] * 0.33 + usefulness[[2]][5] * 0) / popularity
        
        names <- c(names, as.character(gsub("WorkToolsFrequency", "", platform)))
        popularities <- c(popularities, popularity * score)
    }
    
    scores_df <- data.frame(
        Popularity = popularities,
        Name = names
    )
    
    technologies[, i] <- head(as.character((scores_df %>% arrange(desc(Popularity)))$Name), n = 10)
    i <- i + 1
}

technologies

Here, we can see a couple of insightful trends. **Python, SQL, R, Jupyter, Unix, and TensorFlow are all ubiquitous.** However, Spark, Hadoop, and Tableau are unique to data scientists, NoSQL is unique to software engineers, and MATLAB is only really used by researchers and machine learning engineers. While some of these could have been guessed, this is confirmation of the tools that are good to learn for anyone aspiring to have one of these jobs in the future.

## 3.4. Important job methods
<a id="important_job_methods"></a>

Again, we can make the same table as we did for the previous questions, but for methods used on the job rather than skills. Here, we can see the top 10 methods used for all respondents, and then by the same four job titles that we isolated in the last question.

In [ ]:
# Get all column names that begin with "WorkToolsFrequency" and end in a letter
methods <- grep("^WorkMethodsFrequency.*[A-z]$", names(results), value=T)
positions <- c("All", "Data Scientist", "Software Engineer", "Researcher", "Machine Learning Engineer")

technologies <- matrix("", 10, length(positions))
colnames(technologies) <- positions
rownames(technologies) <- 1:10

i <- 1

for (position in positions) {
    names <- c()
    popularities <- c()
    
    if (position == "All") {
        position_results <- results
    } else {
        position_results <- results %>% filter(CurrentJobTitleSelect == position)
    }
    
    for (method in methods) {
        usefulness <- position_results %>%
            group_by_(method) %>%
            count()
        
        # Popularity = the number of people who responded to this question
        popularity <- usefulness[[2]][2] + usefulness[[2]][3] + usefulness[[2]][4] + usefulness[[2]][5]
        
        # Usefulness = a weighted average determining how much this tool was used
        score <- (usefulness[[2]][2] * 1 + usefulness[[2]][3] * 0.67 + usefulness[[2]][4] * 0.33 + usefulness[[2]][5] * 0) / popularity
        
        names <- c(names, as.character(gsub("WorkMethodsFrequency", "", method)))
        popularities <- c(popularities, popularity * score)
    }
    
    scores_df <- data.frame(
        Popularity = popularities,
        Name = names
    )
    
    technologies[, i] <- head(as.character((scores_df %>% arrange(desc(Popularity)))$Name), n = 10)
    i <- i + 1
}

technologies

We can see a number of trends in this table as well. **Everyone is using data visualization, cross validation, logistic regression, and decision trees.** Natural language processing and neural networks are understandably used more often by machine learning engineers, while software engineers are the only ones who often use A/B testing.

# 4. Conclusion
<a id="conclusion"></a>

If you're a novice data scientist and you're just looking for somewhere to get started, the graphs we created strongly support a couple of recommendations:

1. **Learn Python.** Both Python and R have been around for decades, but as we can see in [the first graph](#programming_language), Python won out in basically every scenario. This is also backed up by [the fourth graph](#important_job_skills), in which participants overwhelmingly said that Python was the most important skill on the job. I honestly believe you will be hard-pressed to find a company that doesn't use Python at all, so you should be good to go.
2. **Major in CS and/or Math.** As I mentioned just after creating the [graph of majors vs. job titles](#majors_and_likely_positions), every major has some presence in every role. However, judging from the proportions visible on the bar graph, CS majors and math majors are the most abundant in almost every role. Although it's not a requirement, either or both these majors may give you a bit of an edge.
3. **Make projects, take courses, and participate in Kaggle challenges.** As we could see in [the graph analyzing the best resources for learning](#resources_for_learning), there were a couple of very clear winners. Creating projects, taking online courses, and participating in Kaggle challenges all took the cake as the three most useful resources for learning about data science.
4. **Learn about the most widely used tools.** There seems to be a neverending supply of data science tools and libraries, but this survey allowed us to see the ones that were considered the most important. The most recommended tools were Python, SQL, R, Jupyter, and Unix ([click here to see more](#tools_in_the_real_world)), and the most recommended methods to learn were data visualization, [cross validation](https://en.wikipedia.org/wiki/Cross-validation), [logistic regression](https://en.wikipedia.org/wiki/Logistic_regression), [decision trees](https://en.wikipedia.org/wiki/Decision_tree), and [random forests](https://en.wikipedia.org/wiki/Random_forest) ([click here to see more](#important_job_methods)).

Hopefully this helps you figure out where you should get started! I wouldn't consider myself a professional, but if you have any questions, I'll do my best to answer them in the comments. Thanks for reading!

In [ ]:

{
   "schemaVersion": 2,
   "mediaType": "application/vnd.docker.distribution.manifest.v2+json",
   "config": {
      "mediaType": "application/vnd.docker.container.image.v1+json",
      "size": 22869,
      "digest": "sha256:544e8081ed30d9dc1fb61f0203bd09742e4c044cac9488feeda117427957f672"
   },
   "layers": [
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 50382957,
         "digest": "sha256:7e2b2a5af8f65687add6d864d5841067e23bd435eb1a051be6fe1ea2384946b4"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 222909892,
         "digest": "sha256:59c89b5f9b0c6d94c77d4c3a42986d420aaa7575ac65fcd2c3f5968b3726abfc"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 195204532,
         "digest": "sha256:4017849f9f85133e68a4125e9679775f8e46a17dcdb8c2a52bbe72d0198f5e68"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 1522,
         "digest": "sha256:c8b29d62979a416da925e526364a332b13f8d5f43804ae98964de2a60d47c17a"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 717,
         "digest": "sha256:12004028a6a740ac35e69f489093b860968cc37b9668f65b1e2f61fd4c4ad25c"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 247,
         "digest": "sha256:3f09b9a53dfb03fd34e35d43694c2d38656f7431efce0e6647c47efb5f7b3137"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 408,
         "digest": "sha256:03ed58116b0cb733cc552dc89ef5ea122b6c5cf39ec467f6ad671dc0ba35db0c"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 331594702,
         "digest": "sha256:7844554d9ef75bb3f1d224e166ed12561e78add339448c52a8e5679943b229f1"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 112664904,
         "digest": "sha256:f1dc3596458cc80f32c2e7119ff5e4cf5714cf0f78aea5e1c0e15837c7e59c07"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 413,
         "digest": "sha256:956baf18c9b14836113107d2ff2f0402b4d2019eb0f2f3c14efbd13901a01da8"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 5494,
         "digest": "sha256:bd3207116dc18cbf7caea7b3938f76d796a4ea2976334cbb891f8bd47134e869"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 1839,
         "digest": "sha256:6ae85f598682e81358f2312fb59d5dfba3211a62f18beae32f79b8452635375b"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 2447018295,
         "digest": "sha256:9130dcc5376a37ff1f55ce8782176f810c740728d536cf5be8a8182257f0bc06"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 467,
         "digest": "sha256:4055b5b31384adaf62aa1e8a5ae32c4f9caff7928d2bed7e3ee18f6dc01113a0"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 75218657,
         "digest": "sha256:a668651b432a08004ec11e226c64ac93e8dd2ed634eaa2ac7ffa01606095dea7"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 165682075,
         "digest": "sha256:dab43881d8fb6ac72e1a9b4885ba187c8ded078648cb03d9dc163097ca04430e"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 209796818,
         "digest": "sha256:0d166dce96d949af0ed75d180eae6e939f584526047b92b13da96269cc5fcd45"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 356355573,
         "digest": "sha256:c31249c36c11fd3017dddb6e5b69a9bbdd7508aeff8a8b501283a4fc921c56f7"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 975788160,
         "digest": "sha256:817aedeb45dea437de094eee45a5d43c565fce8cb82b25e4a6215fc9b4df1994"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 1544,
         "digest": "sha256:ac64669d5819066ba1d92a7e62abce4dab782a8593bde2024e3f95be91fa8563"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 902,
         "digest": "sha256:014c20c3e8d7837f3b716b5a7effa65febbe1faa4fdeeea0dc563f4b13d9b84a"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 207,
         "digest": "sha256:0b4224ff3dae732585ceea36367958d49f51102518bf628eb55d60d7f4a2085a"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 372,
         "digest": "sha256:4a8f41188c3cf1da65508940a25dd09881e87adacb1e60d0844b7f86f9e15713"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 972,
         "digest": "sha256:8f8f384875c374e33ec7631efc1a527027aab4c0dff81dce7d1b19103a2b0c79"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 458,
         "digest": "sha256:bb3e64b44abdc3b0d6396b18c187cc0dfe1741a174e93393e9776020c25d8fc2"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 528436201,
         "digest": "sha256:a29c468d2ddf2c1be7a4356fd8b70ddf8b2817b2ad26f3011c194bd2521ebc1d"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 76614148,
         "digest": "sha256:47e01ab28eb7e23d4b257f362b34bfdba6510b83c6e080d64d797617bc1e696a"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 294373,
         "digest": "sha256:a2202eec93ba50fbb865c58f459b179057607fb0afc80f807f053a40b6fe9f81"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 104190808,
         "digest": "sha256:26bfb821987de0d0ec5b91da6895b9fe9a8738d48cd0b0af1f95763e8c67229f"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 684543716,
         "digest": "sha256:211b472bdc79c5725b3889c747116c70b7a2aa33667f58307a75c019f5f1f7d9"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 212,
         "digest": "sha256:9e05ecb0d9efcd65f675ec5463ca07780a5c3e78bee637c02783d2d6c1b70a2a"
      }
   ]
}


In [ ]:

{
   "schemaVersion": 2,
   "mediaType": "application/vnd.docker.distribution.manifest.v2+json",
   "config": {
      "mediaType": "application/vnd.docker.container.image.v1+json",
      "size": 22919,
      "digest": "sha256:e0dda0c528b7c846ad1a9fe316de11aeff89bdbae07f32a5efb11120a3e55ccd"
   },
   "layers": [
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 50382957,
         "digest": "sha256:7e2b2a5af8f65687add6d864d5841067e23bd435eb1a051be6fe1ea2384946b4"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 222909892,
         "digest": "sha256:59c89b5f9b0c6d94c77d4c3a42986d420aaa7575ac65fcd2c3f5968b3726abfc"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 195204532,
         "digest": "sha256:4017849f9f85133e68a4125e9679775f8e46a17dcdb8c2a52bbe72d0198f5e68"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 1522,
         "digest": "sha256:c8b29d62979a416da925e526364a332b13f8d5f43804ae98964de2a60d47c17a"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 717,
         "digest": "sha256:12004028a6a740ac35e69f489093b860968cc37b9668f65b1e2f61fd4c4ad25c"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 247,
         "digest": "sha256:3f09b9a53dfb03fd34e35d43694c2d38656f7431efce0e6647c47efb5f7b3137"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 408,
         "digest": "sha256:03ed58116b0cb733cc552dc89ef5ea122b6c5cf39ec467f6ad671dc0ba35db0c"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 331594702,
         "digest": "sha256:7844554d9ef75bb3f1d224e166ed12561e78add339448c52a8e5679943b229f1"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 112665282,
         "digest": "sha256:f15956f7439a8f32fc517e733c3af3104f414f6fccaf6eb901bcb1e233d4f9bf"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 421,
         "digest": "sha256:04daa549c9f7ce91a4b748f66094533c5b54256c7de57023d09e76728bc55ef1"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 5494,
         "digest": "sha256:ce39292f31e2ce9cd52a97a59962ac74127458ba3cb69a94373cf6aa4116f0c4"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 1838,
         "digest": "sha256:3874427bf61ee09a3145aa945f0036f15da21399e24b84cd84c067e930385979"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 2451236145,
         "digest": "sha256:97b6e7362f6a297643e996bfae1fe57f228e0c47ed322cd8d0f440c0d1e7da3f"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 468,
         "digest": "sha256:5bc4129a64dc88b9963093f616cb5379206ea3e846b7cea67f9142d68b034e39"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 75218781,
         "digest": "sha256:7547bbd42f0a313b4c9f02b490eafc2ebb58447144e4336f7557ed54300d0f06"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 165683068,
         "digest": "sha256:d6d8e4ad22a25bd9f419cd7e654f0d86c3cec95f13ce0af79fca1332a65e1c42"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 209256259,
         "digest": "sha256:011f6e7149f1429ecf5427957e27c2cdb14e4f49267f529f0078c08b85db3e49"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 356432441,
         "digest": "sha256:3c257c57cde2630c7d008673344b11d367d7ac2e815ea381d43da665b11a33ac"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 975781677,
         "digest": "sha256:4660e6449f134a016ecfc33c63503d63d29f221a52416c8f6b49c1072af49f66"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 1545,
         "digest": "sha256:fa3e772df1f71d1992a4dd1a8bc0a3c21be024b379b934f2fdf8c5793f0a9187"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 905,
         "digest": "sha256:12551290e2a447b322755484dafdba1d976252a7d0b33248802fa64fce3f71a2"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 208,
         "digest": "sha256:536ada87436e5c7725482303241b35a7376d00009fe2565bda75acfa4bc89dd6"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 372,
         "digest": "sha256:b7ddb78236b9692ef01e16b35dbbe60ccc6b42cc1c5fee96509843af346d2669"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 972,
         "digest": "sha256:1c45f7a98c9f3768a0b5ed9dd77bbeb1cb7da04be9984e8ef7252b50d01f24d3"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 458,
         "digest": "sha256:6038025defaac44414de10f156ce4689d253166a344670d5a837f837f4b927c1"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 532044270,
         "digest": "sha256:2920d7aee412b4a2892e909fd0119de36d3e835a134cca3c3db035ecd1a81da1"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 74333342,
         "digest": "sha256:6e70c81597fda37fba0af67f986b835420b192885865d9bd87abed01ad44b4a2"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 292263,
         "digest": "sha256:26f5f4c71cf7e877a0afac228e743ea849197aa423a36fcfda7051c8a6f0da82"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 104190576,
         "digest": "sha256:91a9ab9e6ab2972ed7d61aedd2976d7efee1521d1055aa0bc0a8ffa747523d77"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 684547685,
         "digest": "sha256:8087bcd5f1ffed27265ff1c4d1e10fff2ec69bbc35bc7ff76f79deb336bb881d"
      },
      {
         "mediaType": "application/vnd.docker.image.rootfs.diff.tar.gzip",
         "size": 213,
         "digest": "sha256:e4ca55700431faa1342922d093927a79b52db7d817ee4b09fb47e6d9dc22a254"
      }
   ]
}
